In [1]:
from resources.network import Network
import torch
import torch.onnx

In [2]:
model = Network()
model.load_state_dict(torch.load("state_dict_model-Run(lr=0.01, batch_size=1000, shuffle=True, device='cuda').pt", weights_only=True))
model.eval()

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

In [3]:
torch_input = torch.randn(1, 1, 28, 28)
onnx_program = torch.onnx.export(model, torch_input,'ImageClassifier.onnx')

In [4]:
import os
import boto3
from botocore.client import Config
from boto3 import session

In [5]:
key_id = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
bucket = os.environ.get('AWS_S3_BUCKET')
endpoint = os.environ.get('AWS_S3_ENDPOINT')
region = os.environ.get('AWS_DEFAULT_REGION')

#Define client session
session = boto3.session.Session(aws_access_key_id=key_id,aws_secret_access_key=secret_key)

#Define client connection
s3_client = boto3.client('s3', aws_access_key_id=key_id,aws_secret_access_key=secret_key,aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
                        endpoint_url=endpoint,
                        region_name=region)

In [6]:
s3_client.upload_file('ImageClassifier.onnx', bucket, 'ImageClassifier.onnx')

In [8]:
s3_client.list_objects_v2(Bucket=bucket)

{'ResponseMetadata': {'RequestId': 'tx00000724f215108edd1e8-0066f40552-11d1a10-ocs-storagecluster-cephobjectstore',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'transfer-encoding': 'chunked',
   'x-amz-request-id': 'tx00000724f215108edd1e8-0066f40552-11d1a10-ocs-storagecluster-cephobjectstore',
   'content-type': 'application/xml',
   'date': 'Wed, 25 Sep 2024 12:42:58 GMT',
   'connection': 'Keep-Alive'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'ImageClassifier.onnx',
   'LastModified': datetime.datetime(2024, 9, 25, 12, 42, 46, 355000, tzinfo=tzlocal()),
   'ETag': '"c9eb57e904fe0f3cddc502a017bdcae5"',
   'Size': 133758,
   'StorageClass': 'STANDARD'},
  {'Key': 'iris-training-pipeline/5098c189-39a7-4c8c-9db2-65364d394e08/create-dataset/iris_dataset',
   'LastModified': datetime.datetime(2024, 9, 23, 19, 21, 41, 159000, tzinfo=tzlocal()),
   'ETag': '"ae25186fe07a7c1aed40dbc24dab5e4a"',
   'Size': 1809,
   'StorageClass': 'STANDARD'},
  {'Ke